In [30]:
import re

import numpy as np
import pandas as pd
from currency_converter import CurrencyConverter

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoLars
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

from sklearn.metrics import explained_variance_score
from sklearn.metrics import max_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.decomposition import PCA
from sklearn.pipeline import FeatureUnion, Pipeline


## Load & prepare data
1. Load Car4You dataset
2. Load Kaggle dataset: https://www.kaggle.com/datasets/adityadesai13/used-car-dataset-ford-and-mercedes?select=skoda.csv
3. Merge datasets

In [31]:
df_car4you_raw = pd.read_csv('./data/car4you_suv_all_final.csv', delimiter=';')
df_car4you_raw.head()

,web-scraper-order,web-scraper-start-url,price_raw,marke_modell_raw,baujahr_km_treibstoff_getriebe_raw,properties_raw,garage_raw,garage_place_raw,description_raw
0,1652200447-14927,https://www.carforyou.ch/de/auto/suv?page=1205,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1652197876-3840,https://www.carforyou.ch/de/auto/suv?page=1667,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1652201197-18131,https://www.carforyou.ch/de/auto/suv?page=1071,CHF 80’500,Audi Q5 Sportback 45 TFSI Black Edition quattr...,2021 · 2’900 km · Elektro/Benzin · Automatik,pricecheckMarktpreis,NaN,NaN,NaN
3,1652197512-2221,https://www.carforyou.ch/de/auto/suv?page=1734,CHF 39’990,Audi Q2 35 TDI quattro S-tronic,2020 · 10’000 km · Diesel · Automatik,pricecheckÜber dem Marktpreis,NaN,NaN,[YS] schwarz - felsgrau[GB1] LTE-Unterstützung...
4,1652204391-31474,https://www.carforyou.ch/de/auto/suv?page=515,CHF 73’000,Jaguar E-Pace 2.0 I4 200 R-Dynamic Black AWD,2022 · 45 km · Elektro/Benzin · Automatik,pricecheckÜber dem MarktpreisumbrellaKäuferschutz,Emil Frey Zürich Altstetten,"8048 Zürich, Zürich",NaN


In [62]:
from typing import Dict
import re


def extract_regex(pattern: str, target: str) -> str:
    if not pd.isnull(target) and re.search(pattern, target):
        return re.findall(pattern, target)[0]
    return np.NaN


def extract_manufacturer(cell_content) -> str:
    if not pd.isnull(cell_content):
        return cell_content.split(' ')[0]
    return np.NaN


def extract_year(cell_content: str) -> int:
    year_str = extract_regex(r'[12][0-9]{3}', cell_content)
    return np.NaN if pd.isnull(year_str) else int(year_str)


def extract_km(cell_content: str) -> int:
    km_str = extract_regex(r'\d*’*\d+ km', cell_content)
    if pd.isnull(km_str):
        return np.NaN
    km = km_str.replace('’', '').replace(' km', '')
    return int(km)


def extract_fuel(cell_content: str) -> str:
    if "Elektro/Benzin" in cell_content or "Elektro/Diesel" in cell_content:
        return "Hybrid"
    if "Benzin" in cell_content:
        return "Petrol"
    if "Diesel" in cell_content:
        return "Diesel"
    if "Elektro" in cell_content:
        return "Electric"

    return np.NaN

def extract_transmission(cell_content: str) -> str:
    if "Automatik" in cell_content:
        return "Automatic"
    if "Manuell" in cell_content:
        return "Manual"
    return np.NaN

def extract_price(cell_content: str) -> str:
    if pd.isnull(cell_content):
        return np.NaN
    return str(cell_content).replace('CHF ', '').replace('’', '')

def extract_price_check(cell_content: str) -> str:
    if "Über dem Marktpreis" in cell_content:
        return "Above market"
    if "Unter dem Marktpreis" in cell_content:
        return "Below market"
    if "Marktpreis" in cell_content:
        return "Market"
    return np.NaN

def row_to_dict(row: pd.Series) -> Dict:
    row_dict = {
        'manufacturer': extract_manufacturer(row[3]),
        'year': extract_year(str(row[4])),
        'km': extract_km(str(row[4])),
        'fuel': extract_fuel(str(row[4])),
        'transmission': extract_transmission(str(row[4])),
        'price': extract_price(row[2]),
        'price_check': extract_price_check(str(row[5])),
    }

    return row_dict


In [63]:
data_raw = []
for index, row in df_car4you_raw.iterrows():
    data_raw.append(row_to_dict(row))

df_car4you = pd.DataFrame(data_raw)
df_car4you.head()

,manufacturer,year,km,fuel,transmission,price,price_check
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Audi,2021.0,2900.0,Hybrid,Automatic,80500,Market
3,Audi,2020.0,10000.0,Diesel,Automatic,39990,Above market
4,Jaguar,2022.0,45.0,Hybrid,Automatic,73000,Above market


In [51]:
curr_converter = CurrencyConverter('./data/eurofxref.csv')


def load_kaggle_dataset(path: str, manufacturer: str) -> pd.DataFrame:
    df_raw = pd.read_csv(path).drop(columns=['model', 'tax', 'tax(£)', 'mpg', 'engineSize'], errors='ignore')
    df_raw.insert(0, 'manufacturer', manufacturer)
    df_raw['price_check'] = 'Market'
    df_raw.price = df_raw.price.apply(lambda x: curr_converter.convert(x, 'GBP', 'CHF'))
    df_raw.mileage = df_raw.mileage * 1.61
    return df_raw.rename(columns={'mileage': 'km', 'fuelType': 'fuel'})

In [52]:
df_audi = load_kaggle_dataset('./data/audi.csv', 'Audi')
df_bmw = load_kaggle_dataset('./data/bmw.csv', 'BMW')
df_merc_c_class = load_kaggle_dataset('./data/cclass.csv', 'Mercedes-Benz')
df_merc = load_kaggle_dataset('./data/merc.csv', 'Mercedes-Benz')
df_ford_focus = load_kaggle_dataset('./data/focus.csv', 'Ford')
df_ford = load_kaggle_dataset('./data/ford.csv', 'Ford')
df_hyundai = load_kaggle_dataset('./data/hyundi.csv', 'Hyundai')
df_skoda = load_kaggle_dataset('./data/skoda.csv', 'Skoda')
df_toyota = load_kaggle_dataset('./data/toyota.csv', 'Toyota')
df_vauxhall = load_kaggle_dataset('./data/vauxhall.csv', 'Vauxhall')
df_vw = load_kaggle_dataset('./data/vw.csv', 'VW')

In [64]:
df_raw = pd.concat([df_car4you, df_audi, df_bmw, df_merc_c_class, df_merc, df_ford_focus, df_ford, df_hyundai, df_skoda, df_toyota, df_vauxhall, df_vw])
df_raw.head()

,manufacturer,year,km,fuel,transmission,price,price_check
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Audi,2021.0,2900.0,Hybrid,Automatic,80500,Market
3,Audi,2020.0,10000.0,Diesel,Automatic,39990,Above market
4,Jaguar,2022.0,45.0,Hybrid,Automatic,73000,Above market


In [54]:
df_raw.manufacturer.value_counts()

Ford             24752
Mercedes-Benz    19699
VW               17877
BMW              13968
Vauxhall         13632
Audi             13129
Toyota            7962
Skoda             7117
Hyundai           5852
Land              1563
Jeep              1528
Volvo             1340
Opel              1103
Peugeot            974
Mazda              940
Nissan             865
Suzuki             839
Porsche            812
Subaru             688
Renault            666
SEAT               665
Kia                616
Mitsubishi         533
Mini               395
Citroën            383
Alfa               380
Jaguar             375
CUPRA              355
Honda              322
Dacia              292
Lexus              206
SsangYong          199
Dodge              191
Fiat               172
DS                 154
JAC                127
Cadillac           111
Chevrolet          106
Maserati           101
Isuzu               62
Infiniti            48
Tesla               46
Hummer              41
Daihatsu   

In [60]:
df_raw.fuel.value_counts()

Petrol      74301
Diesel      55135
Hybrid       9806
Electric     1817
Other         253
Name: fuel, dtype: int64

In [65]:
df_raw.transmission.value_counts()

Manual       65739
Automatic    48836
Semi-Auto    24903
Other           10
Name: transmission, dtype: int64

In [67]:
print(f'Length before dropna(): {len(df_raw)}')
df_raw = df_raw.dropna()
print(f'Length after dropna(): {len(df_raw)}')

Length before dropna(): 152437
Length after dropna(): 121524


In [70]:
df = df_raw.astype({
    'manufacturer': 'category',
    'year': 'int32',
    'km': 'float',
    'fuel': 'category',
    'transmission': 'category',
    'price':'float',
    'price_check': 'category'
})
df.head()

,manufacturer,year,km,fuel,transmission,price,price_check
2,Audi,2021,2900.0,Hybrid,Automatic,80500.0,Market
3,Audi,2020,10000.0,Diesel,Automatic,39990.0,Above market
4,Jaguar,2022,45.0,Hybrid,Automatic,73000.0,Above market
7,VW,2021,23336.0,Petrol,Automatic,29900.0,Market
9,BMW,2019,20400.0,Diesel,Automatic,36900.0,Market


In [71]:
df.dtypes

manufacturer    category
year               int32
km               float64
fuel            category
transmission    category
price            float64
price_check     category
dtype: object

In [72]:
df.describe()

,year,km,price
count,121524.000000,121524.000000,121524.000000
mean,2017.178269,38626.567562,22769.512632
std,2.303724,35296.025976,14991.364453
min,1970.000000,1.000000,549.051284
25%,2016.000000,12186.090000,12870.372144
50%,2017.000000,28947.800000,18950.810080
75%,2019.000000,54494.072500,28050.420020
max,2060.000000,520030.000000,259900.000000


In [73]:
df.manufacturer = df.manufacturer.cat.codes
df.transmission = df.transmission.cat.codes
df.fuel = df.fuel.cat.codes
df.price_check = df.price_check.cat.codes

In [74]:
df.price_check.value_counts()

1    116123
0      5401
Name: price_check, dtype: int64

## Outlier analysis

## Data preparation

In [75]:
X = df.drop(columns='price')
y = df.price


## Price prediction

In [76]:
class BenchmarkRegressor:
    def __init__(self):
        pass

    def fit(self, X, y, **kwargs):
        self.mean = y.mean()

    def predict(self, X):
        return [self.mean] * len(X)

    def get_params(self, deep=False):
        return {}

In [77]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=21, stratify=df.price_check)

In [78]:
class BenchmarkRegressor:
    def __init__(self):
        pass

    def fit(self, X, y, **kwargs):
        self.mean = y.mean()

    def predict(self, X):
        return [self.mean] * len(X)

    def get_params(self, deep=False):
        return {}

# The following Models are currently computing used the Standard Parameters. Please keep in mind,
# that they need to be optimized using the hyperparameters within the modeling process.

bm_regr = BenchmarkRegressor()
lr_regr = LinearRegression()
ri_regr = Ridge()
br_regr = BayesianRidge()
ls_regr = Lasso()
ll_regr = LassoLars()
en_regr = ElasticNet()
ne_regr = KNeighborsRegressor()
dt_regr = DecisionTreeRegressor()
rf_regr = RandomForestRegressor()
xg_regr = xgb.XGBRegressor()

models = [
    ('Benchmark', bm_regr),
    ('LR', lr_regr),
    ('Ridge', ri_regr),
    ('Bayesian Ridge', br_regr),
    ('Lasso', ls_regr),
    ('LARS Lasso', ll_regr),
    ('Elastic Net', en_regr),
    ('Nearest Neighbors (KNN) regression', ne_regr),
    ('Decision Tree', dt_regr),
    ('Random Forest', rf_regr),
    ('XGBoost', xg_regr),
]

In [79]:

for name, model in models:
    pipelined_model = Pipeline([
        ('pca', PCA(n_components = 4)),
        (name, model)
    ])

    # Training des Models
    pipelined_model.fit(X_train, y_train)

    # Vorhersage des trainierten models auf X_test
    y_hat = pipelined_model.predict(X_test)

    # Berechnung der verschiedenen Messwerte resp. KPI's
    EVS = (explained_variance_score(y_test, y_hat))
    ME = (max_error(y_test, y_hat))
    MAE = (mean_absolute_error(y_test, y_hat))
    MSE = np.sqrt(mean_squared_error(y_test, y_hat, squared=True))
    RMSE = np.sqrt(mean_squared_error(y_test, y_hat, squared=False))
    R2 = r2_score(y_test, y_hat)
    print('Model: ', name, ' | EVS: ', EVS)
    print('Model: ', name, ' | ME: ', ME)
    print('Model: ', name, ' | MAE: ', MAE)
    print('Model: ', name, ' | MSE ', MSE)
    print('Model: ', name, ' | RMSE ', RMSE)
    print('Model: ', name, ' | R2 ', R2)
    print('----------------')


Model:  Benchmark  | EVS:  2.220446049250313e-16
Model:  Benchmark  | ME:  227138.12594660855
Model:  Benchmark  | MAE:  10400.27114315184
Model:  Benchmark  | MSE  15067.991207570254
Model:  Benchmark  | RMSE  122.75174625059415
Model:  Benchmark  | R2  -6.4246579267734916e-06
----------------
Model:  LR  | EVS:  0.2869564624807597
Model:  LR  | ME:  224455.52638555816
Model:  LR  | MAE:  8169.171119461458
Model:  LR  | MSE  12723.658515065552
Model:  LR  | RMSE  112.79919554263475
Model:  LR  | R2  0.2869564044838522
----------------
Model:  Ridge  | EVS:  0.2869564830091914
Model:  Ridge  | ME:  224455.5246335601
Model:  Ridge  | MAE:  8169.170583269668
Model:  Ridge  | MSE  12723.658331921668
Model:  Ridge  | RMSE  112.79919473082096
Model:  Ridge  | R2  0.2869564250108999
----------------
Model:  Bayesian Ridge  | EVS:  0.2869575351040806
Model:  Bayesian Ridge  | ME:  224455.4343554694
Model:  Bayesian Ridge  | MAE:  8169.1429742716655
Model:  Bayesian Ridge  | MSE  12723.6489456

c:\users\yvesh\.virtualenvs\data-science-22-car4you\lib\site-packages\sklearn\linear_model\_base.py:133: FutureWarning: The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LassoLars())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  warnings.warn(


Model:  Elastic Net  | EVS:  0.28441743738324965
Model:  Elastic Net  | ME:  224367.0547997509
Model:  Elastic Net  | MAE:  8176.108240681918
Model:  Elastic Net  | MSE  12746.292346528084
Model:  Elastic Net  | RMSE  112.89947894710623
Model:  Elastic Net  | R2  0.284417313526812
----------------
Model:  Nearest Neighbors (KNN) regression  | EVS:  0.3525064305739556
Model:  Nearest Neighbors (KNN) regression  | ME:  209816.0
Model:  Nearest Neighbors (KNN) regression  | MAE:  7789.025435285631
Model:  Nearest Neighbors (KNN) regression  | MSE  12128.251695764515
Model:  Nearest Neighbors (KNN) regression  | RMSE  110.1283419277913
Model:  Nearest Neighbors (KNN) regression  | R2  0.3521290955868309
----------------
Model:  Decision Tree  | EVS:  0.537607368783831
Model:  Decision Tree  | ME:  210969.41706318903
Model:  Decision Tree  | MAE:  5368.124933095986
Model:  Decision Tree  | MSE  10246.216069286214
Model:  Decision Tree  | RMSE  101.22359442978804
Model:  Decision Tree  | R2 